In [1]:
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

'3.6.3'

In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [3]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [4]:
import logging

class PlanetVBSpider(scrapy.Spider):
    name = "planetvb"
    start_urls = [
        'http://www.planetvb.com/modules/newbb/'
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'quoteresult.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for line in response.xpath('//div[@id="index_forum"]'):
            
            forum_title = line.css('b::text').extract_first()
            #subtitle = line.xpath('./font[@size="2"]/text()').extract_first()
            url_threads = line.xpath('./a/@href').extract_first()
            
            yield scrapy.Request(url = url_threads, 
                                 callback = self.parse_sublevel, 
                                 meta = {'forum_title': forum_title})
    
    def parse_sublevel(self, response):
        forum_title = response.meta['forum_title']
        for line in response.xpath('//a[starts-with(@href, "viewtopic") and not(contains(@href, "start"))]'):
            url_thread = line.css('a::attr(href)').extract_first()
            meta_sublevel = {
            'forum_code': url_thread[url_thread.find("&forum=")+7:],
            'forum_title': forum_title, 
            'thread_code': url_thread[url_thread.find("topic_id=")+9:url_thread.find("&forum")],
            'thread_title': line.css('a::text').extract_first()[5:]    # Todos empiezan por un feo \r\n\t\t\t
            }
            
            # AFEGIR AUTOR!!!
            
            url_thread_abs = response.urljoin(url_thread)
            
            yield scrapy.Request(url = url_thread_abs, 
                                 callback=self.parse_thread, meta= meta_sublevel)
            
        # FOLLOW PAGINATION
        if response.xpath('//a[starts-with(text(),"»")]'):
            next_thread_page_url = 'http://www.planetvb.com' + response.xpath('//a[starts-with(text(),"»")]').css('a::attr(href)')[0].extract()
            yield scrapy.Request(url = next_thread_page_url, 
                                 callback = self.parse_sublevel, 
                                 meta = response.meta)

        
    def parse_thread(self, response):
        
        forum_code = response.meta['forum_code']
        forum_title = response.meta['forum_title']
        thread_code = response.meta['thread_code']
        thread_title = response.meta['thread_title']
        
        for line in response.xpath('//div[@class = "forum-post"]'):
            yield {  
            'forum_code': forum_code,
            'forum_title': forum_title, 
            'thread_code': thread_code,
            'thread_title': thread_title,
            'author': line.xpath('./div/div/select/option/text()').extract_first(),
            'post_date': line.xpath('./div/div/span[@class="forum-post-body-posted-on-date"]/text()').extract_first(),
            'post_text': line.xpath('normalize-space(./div/div[@class = "forum-post-body-right"])').extract_first()   
            }
        
        # FOLLOW PAGINATION
        if response.xpath('//a[starts-with(text(),"»")]'):
            next_thread_page_url = 'http://www.planetvb.com' + response.xpath('//a[starts-with(text(),"»")]').css('a::attr(href)')[0].extract()
            yield scrapy.Request(url = next_thread_page_url, 
                                 callback = self.parse_thread, 
                                 meta = response.meta)
        
                
        # USER CENTER PER DATOS USER


In [5]:
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

process = CrawlerProcess(get_project_settings())

process.crawl(PlanetVBSpider)
process.start()

2018-01-24 18:28:03 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2018-01-24 18:28:03 [scrapy.utils.log] INFO: Overridden settings: {}


In [6]:
import pandas as pd
dfjson = pd.read_json('quoteresult.json')
dfjson

,author,forum_code,forum_title,post_date,post_text,thread_code,thread_title
0,KGRAMR,1,Main Virtual Boy Discussion,1/18 22:44,I was browsing Twitter when all of the sudden ...,6982,Invincible Iron Man Gagaga-In (Planned 8ing VB...
1,Benjamin Stevens,1,Main Virtual Boy Discussion,1/18 23:49,That is a very interesting find! Looks like we...,6982,Invincible Iron Man Gagaga-In (Planned 8ing VB...
2,KGRAMR,1,Main Virtual Boy Discussion,1/19 0:49,Quote:Benjamin Stevens wrote:That is a very in...,6982,Invincible Iron Man Gagaga-In (Planned 8ing VB...
3,Benjamin Stevens,1,Main Virtual Boy Discussion,1/19 1:54,The text on the page reads as follows:A giant ...,6982,Invincible Iron Man Gagaga-In (Planned 8ing VB...
4,Dreammary,1,Main Virtual Boy Discussion,1/19 3:04,"That’s really cool, it’s unfortunate that the ...",6982,Invincible Iron Man Gagaga-In (Planned 8ing VB...
5,vuefinder83,1,Main Virtual Boy Discussion,1/19 3:48,"Nice find there, KGRAMR! Quote:Benjamin Steven...",6982,Invincible Iron Man Gagaga-In (Planned 8ing VB...
6,KGRAMR,1,Main Virtual Boy Discussion,1/19 3:51,Quote:Benjamin Stevens wrote:The text on the p...,6982,Invincible Iron Man Gagaga-In (Planned 8ing VB...
7,KR155E,1,Main Virtual Boy Discussion,1/19 11:30,Very cool discovery! I did a bit of research o...,6982,Invincible Iron Man Gagaga-In (Planned 8ing VB...
8,Bomberman94,1,Main Virtual Boy Discussion,1/19 12:22,Has someone stored the video and is able to sh...,6982,Invincible Iron Man Gagaga-In (Planned 8ing VB...
9,astro187,1,Main Virtual Boy Discussion,1/19 15:10,What to potential awesome discovery! Maybe our...,6982,Invincible Iron Man Gagaga-In (Planned 8ing VB...


In [7]:
dfjson.to_csv('planetVB_posts_time.csv', index=False)

In [9]:
json.dumps('quoteresult.json')

'"quoteresult.json"'

In [10]:
dfjson['thread_title'][0][5:]

'The Noteworthy Auction Thread™'

In [73]:
topic = 'viewtopic.php?topic_id=15845&forum=1'
topic[topic.find("topic_id=")+9:topic.find("&forum")]

'15845'

In [72]:
topic[topic.find("&forum=")+7:]

'1'

In [14]:
dfjson[dfjson['forum_title']=='Main Virtual Boy Discussion']

,forum_title,thread_title,url_thread
0,Main Virtual Boy Discussion,The Noteworthy Auction Thread™,viewtopic.php?topic_id=158&forum=1
1,Main Virtual Boy Discussion,Virtual Boy Serial Number Inventory,viewtopic.php?topic_id=3127&forum=1
2,Main Virtual Boy Discussion,Virtual Boy Screen Repairs,viewtopic.php?topic_id=6885&forum=1
37,Main Virtual Boy Discussion,Invincible Iron Man Gagaga-In (Planned 8ing VB...,viewtopic.php?topic_id=6982&forum=1
38,Main Virtual Boy Discussion,New member - hello,viewtopic.php?topic_id=6980&forum=1
39,Main Virtual Boy Discussion,Provenance of Blockbuster VB rental ad?,viewtopic.php?topic_id=6984&forum=1
40,Main Virtual Boy Discussion,Post pics of your VB Collection,viewtopic.php?topic_id=4678&forum=1
41,Main Virtual Boy Discussion,Virtualboy game made by eighting,viewtopic.php?topic_id=6521&forum=1
42,Main Virtual Boy Discussion,GamePro en Español,viewtopic.php?topic_id=6932&forum=1
43,Main Virtual Boy Discussion,GAME Data Room > VIRTUAL BOY Soft. JPN Release...,viewtopic.php?topic_id=6983&forum=1


In [61]:
forums = dfjson[['forum_title', 'url_thread']]

In [62]:
forums['forum_code'] = forums['url_thread'].str.split('=').str.get(-1)

In [63]:
forums = forums[['forum_title', 'forum_code']]

In [66]:
forums = forums.drop_duplicates()

In [67]:
forums

,forum_title,forum_code
0,Main Virtual Boy Discussion,1
3,Marketplace,3
17,Virtual Boy Development Board,2
60,Offtopic,4
80,PVB Coding Competition 2010,11
100,PVB Coding Competition 2008,8
120,FlashBoy,7
140,VB Dev Repository,9
145,PVB Coding Competition 2013,13
174,Feedback,6


In [68]:
forums.to_csv('forum_codes.csv', index=False)